# COGS 108 - Data Checkpoint

# Names

- Joonsung Park
- James Chung
- Richard Gross
- Madison Hambly
- Colin Lintereur

<a id='research_question'></a>
# Research Question

How do the most popular topics from previous COGS 108 final projects vary per quarter from 2019 to 2021? Also what other trends can we find in past submissions, like changes in word count, and number of graphs used?

# Dataset(s)

- Dataset Name: Past COGS108 final projects
- Link to the dataset: [github.com/COGS108](github.com/COGS108)
- Number of observations: ~200 so far

We looked through the COGS108 github repositories and downloaded the zip file for each quarter of submissions we wanted to include in out analyis. From there, we loop through each jupyter notebook in the zip file and collect relevant data. We also parse the notebook to find the research of the group. We then generate a csv file for each quarter, and once we have all the csv files we of each quarter, we combine them into the final dataset we want to study. There are more quarters and submissions that are not yet included but we plan to include more.

# Setup

In [1]:
# importing libraries
import numpy as np
import json
import zipfile
import pandas as pd
import re
import paralleldots
import seaborn as sns

After importing libraries, we now search through the zip file of a quarter we want to study. Some data we can find through regex of the filename and some we need to parse the notebook to find. In particular, the research question is difficult to find since the formatting of every submission is not always the same. This may cause the parser to not find any research question or find the wrong cell. However, this is very rare and does not actually ruin our analysis since all we need is a block of text that describes the topic of the project and any block of text near the research question tends to contain relevant information to the topic of the project.

In [2]:
# setting API key for classifier
paralleldots.set_api_key("aSWrD1QSsUqdb8JYWJ6jMJMXrTxjFXKhdD4xQZKiUlQ")

# initializing lists
research_questions = {}
qtr = []
year = []
group = []
special = []
word_counts = []
index = 0

# reading zip file for a certain quarter
with zipfile.ZipFile("zips/FinalProjects-SP21-main.zip", "r") as f:
    for i, name in enumerate(f.namelist()):
        # skip files that are not jupyter notebooks
        if '.ipynb' not in name:
            continue
        rf = json.loads(f.read(name))
        word_count = 0
        rq_found = False
        
        # loop through every cell
        for j, cell in enumerate(rf['cells']):
            if cell['cell_type'] == 'markdown':
                for c in cell['source']:
                    # if cell contains the research question header, update lists and extract the research question from the next cell
                    if (rq_found == False) and (re.search(r"(^#.*research question)", c.lower()) != None):
                        #print("RQ FOUND")
                        qtr.append("SP")
                        year.append(2021)
                        group.append(int(re.search(r"[$0-9^]{3,}", name)[0]))
                        special.append(False if re.search(r"_S\.ipynb", name) == None else True)
                        rq = ' '.join(rf['cells'][j+1]['source'])
                        research_questions[str(group[index])+"_"+qtr[index]+"_"+str(year[index])] = rq          
                        rq_found = True
                        index += 1
                    word_count += len(c.replace('#', '').lstrip().split(' '))
        if rq_found:
            word_counts.append(word_count)

After collecting all the data we need from the submissions, we can use a classifier to find the general topic of the submission. We are currently using an API made by ParallelDots to do this classification for us.

In [3]:
# initialize topics list
topic = []

# loop through research questions and find the most confident topic class from topic classifier
for i, (g, q, y) in enumerate(zip(group, qtr, year)):
    response = paralleldots.taxonomy(research_questions[str(group[i])+"_"+qtr[i]+"_"+str(year[i])])
    topic.append(response['taxonomy'][0]['tag'])
    if i == 0:
        print("EXAMPLE:\n", research_questions[str(group[i])+"_"+qtr[i]+"_"+str(year[i])], " :: ", topic[i])

EXAMPLE:
 Which factor, among major, college quality, internship experience, and gender, is the most influential on the starting salary of college graduates in the United States?  ::  EDUCATION


Now that we have stored the topics of the submissions, we can run a sanity check to make sure that all of our list are the same length and then export the data to a csv file.

In [4]:
# double check that all the lists are the same length
print(len(group), len(qtr), len(year), len(topic), len(special), len(word_counts))

# generate data frame and output it to a csv file
d = {'Group':group, 'Quarter':qtr, 'Year':year, 'Topic':topic, 'Special':special, 'Word_Count':word_counts}
df = pd.DataFrame(data = d)
output = df.to_csv(path_or_buf="./"+qtr[0] + str(year[0])+".csv")

47 47 47 47 47 47


# Data Cleaning

Since we are collecting the data ourselves, there is not much work to be done cleaning the data after collecting all of the csv files for each quarter and merging them together into one large dataset. One column we do need to remove however is a column named 'Unnamed: 0'. This column is an artifact of reading csv files that store index information for each dataset. This column is redundant in our new dataset so we can remove it.

In [18]:
# collecting all dataframes together
df_SP21 = pd.read_csv("SP2021.csv")
df_WI21 = pd.read_csv("WI2021.csv")
df_FA20 = pd.read_csv("FA2020.csv")
df_SP20 = pd.read_csv("SP2020.csv")

df_total = df_SP21.merge(df_WI21.merge(df_FA20.merge(df_SP20, how='outer'), how='outer'), how='outer')
df_total = df_total.drop('Unnamed: 0', axis=1)
assert(len(df_total.index) == (len(df_SP21.index)+len(df_WI21.index)+len(df_FA20.index)+len(df_SP20.index)))

df_total.head()

,Group,Quarter,Year,Topic,Special,Word_Count
0,2,SP,2021,EDUCATION,False,4399
1,3,SP,2021,EDUCATION,False,4065
2,4,SP,2021,POLITICS,False,3479
3,5,SP,2021,IMPACT,False,2691
4,6,SP,2021,POLITICS,False,5117
